<a href="https://colab.research.google.com/github/rgreenfield/Song_Lyrics_Generator/blob/main/LyricsGerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import tensorflow as tf
import numpy as np
import os
import time
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku



In [8]:
print(tf. __version__)

2.9.2


In [26]:
path_to_file = tf.keras.utils.get_file('Juice_Wrld_Lyrics', 
      'https://raw.githubusercontent.com/rgreenfield/Song_Lyrics_Generator/main/Juice_Wrld_Lyrics')
      

In [27]:
tokenizer = Tokenizer()
data = open(path_to_file, 'rb').read().decode(encoding='utf-8')
corpus = data.lower().split("\n")

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1
print(tokenizer.word_index)
print(total_words)

{'me': 1, 'with': 2, 'my': 3, 'i': 4, 'the': 5, "i'm": 6, 'drugs': 7, 'but': 8, 'and': 9, 'high': 10, 'if': 11, 'pop': 12, 'you': 13, 'smoke': 14, 'up': 15, 'bad': 16, 'oh': 17, "don't": 18, 'lean': 19, 'get': 20, 'fucked': 21, 'kidneys': 22, 'no': 23, 'whip': 24, 'rock': 25, 'drink': 26, 'liver': 27, 'some': 28, 'so': 29, 'hurt': 30, 'it': 31, 'too': 32, 'not': 33, 'a': 34, 'off': 35, "swervin'": 36, 'please': 37, 'urge': 38, 'know': 39, 'yeah': 40, 'phone': 41, 'to': 42, 'drive': 43, "won't": 44, 'show': 45, 'mercy': 46, 'yuh': 47, 'got': 48, "lookin'": 49, 'on': 50, 'die': 51, 'for': 52, 'love': 53, 'pills': 54, 'right': 55, 'huh': 56, 'turned': 57, 'whole': 58, 'different': 59, 'person': 60, 'crash': 61, 'bitch': 62, 'all': 63, "screamin'": 64, 'these': 65, 'hoes': 66, 'at': 67, 'is': 68, 'her': 69, 'real': 70, 'fuck': 71, 'two': 72, 'ring': 73, 'wrong': 74, 'when': 75, 'where': 76, 'belong': 77, 'they': 78, 'now': 79, 'sticks': 80, 'stones': 81, 'may': 82, 'break': 83, 'bones': 84

In [28]:
# create input sequences using list of tokens
input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

In [29]:
# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
label = ku.to_categorical(label, num_classes=total_words)

In [30]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
#earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
history = model.fit(predictors, label, epochs=50, verbose=1)
#print model.summary()
print(model)

Epoch 1/50
19/19 [==============================] - 6s 47ms/step - loss: 4.8294 - accuracy: 0.0862
Epoch 2/50
19/19 [==============================] - 1s 45ms/step - loss: 4.0625 - accuracy: 0.1534
Epoch 3/50
19/19 [==============================] - 1s 45ms/step - loss: 3.1573 - accuracy: 0.3155
Epoch 4/50
19/19 [==============================] - 1s 46ms/step - loss: 2.2881 - accuracy: 0.4759
Epoch 5/50
19/19 [==============================] - 1s 47ms/step - loss: 1.5537 - accuracy: 0.6207
Epoch 6/50
19/19 [==============================] - 1s 48ms/step - loss: 0.9795 - accuracy: 0.7379
Epoch 7/50
19/19 [==============================] - 1s 46ms/step - loss: 0.6639 - accuracy: 0.8293
Epoch 8/50
19/19 [==============================] - 1s 45ms/step - loss: 0.4142 - accuracy: 0.8948
Epoch 9/50
19/19 [==============================] - 1s 45ms/step - loss: 0.2333 - accuracy: 0.9448
Epoch 10/50
19/19 [==============================] - 1s 44ms/step - loss: 0.1668 - accuracy: 0.9569
Epoch 11/

In [33]:
seed_text = "I'm the definition of a bandit"
next_words = 40
  
for i in range(next_words):
  token_list = tokenizer.texts_to_sequences([seed_text])[0]
  token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
  #predicted = model.predict_classes(token_list, verbose=0)
  predicted = np.argmax(model.predict(token_list), axis=-1)
  output_word = ""
  #print(predicted)
  for word,index in tokenizer.word_index.items():
    #print(word)
    if index == predicted:
      output_word = word
      break
  seed_text += " " + output_word
print(seed_text)

1/1 [==============================] - 0s 23ms/step
I'm the definition of a bandit trouble so i know i'm gonna find it on my own clean it shoulder it when i'm not wrong and shop with me so i'm not worried worried won't hurt me yuh yuh but me the drugs won't yuh yuh


In [34]:
print(seed_text)

I'm the definition of a bandit trouble so i know i'm gonna find it on my own clean it shoulder it when i'm not wrong and shop with me so i'm not worried worried won't hurt me yuh yuh but me the drugs won't yuh yuh


In [ ]:
import pyttsx3

class TextToSpeech:
    engine: pyttsx3.Engine

    def __init__(self, voice, rate: int, volume: float):
        self.engine = pyttsx3.init()
        if voice:
            self.engine.setProperty('voice', voice)
        self.engine.setProperty('rate', rate)
        self.engine.setProperty('volume', volume)  # Between 0 and 1

    def text_to_speech(self, text: str, save: bool = False, file_name='output.mp3'):
        self.engine.say(text)
        print('I\'m speaking...')

        if save:
            # On linux make sure that 'espeak' and 'ffmpeg' are installed
            self.engine.save_to_file(text, file_name)

        self.engine.runAndWait()

    def list_available_voices(self):
        voices: list = [self.engine.getProperty('voices')]

        for i, voice in enumerate(voices[0]):
            print(f'({i + 1}) {voice.name} {voice.age}: {voice.languages[0]} ({voice.gender}) [ID: {voice.id}]')


if __name__ == '__main__':
    
    tts = TextToSpeech('com.apple.speech.synthesis.voice.kyoko', 200, 1.0)
    # tts.list_available_voices()
    tts.text_to_speech(seed_text, save=True, file_name='output.mp3')